In [1]:
import re
import os
import torch
import glob
import hashlib

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from more_itertools import chunked


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
raw_data_folder = "../TEST"
model_name = "intfloat/multilingual-e5-base"
persist_directory = "../chroma_db"

## READ_FILES

In [3]:
txt_files = glob.glob(os.path.join(raw_data_folder, "*.txt"))
md_files = glob.glob(os.path.join(raw_data_folder, "*.md"))

def read_txt_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    
def clean_text(text):
    #text.to_lower()  # Chuyển đổi chữ hoa thành chữ thường
    #text = re.sub(r'#* ', '', text)  # Remove Markdown headers
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = text.strip() 
    return text

markdown_text = ""

for file_path in md_files:
    try:
        content = read_txt_file(file_path)
        #content = clean_text(content)
        if content:
            markdown_text += f"\n\n## {os.path.basename(file_path)}\n\n{content}"
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

In [4]:
loeader = DirectoryLoader(raw_data_folder,
                           glob="**/*.md",
                             show_progress=True,
                              loader_cls=TextLoader,
                              loader_kwargs={"encoding": "utf-8"}
                              )
documents = loeader.load()
len(documents)
documents



100%|██████████| 12/12 [00:00<00:00, 3878.83it/s]


[Document(metadata={'source': '..\\TEST\\file_11_svd.md'}, page_content='# Phân tích giá trị kỳ dị (SVD)\n\n## Định nghĩa\nPhân tích giá trị kỳ dị (Singular Value Decomposition - SVD) là phép phân tích một ma trận \\( A \\) bất kỳ thành ba ma trận:\n\\[ A = U \\Sigma V^T \\]\nTrong đó:\n- \\( U \\) là ma trận trực giao \\( m \\times m \\)\n- \\( \\Sigma \\) là ma trận đường chéo \\( m \\times n \\) chứa các giá trị kỳ dị\n- \\( V^T \\) là chuyển vị của ma trận trực giao \\( n \\times n \\)\n\n## Ý nghĩa hình học\nSVD biểu diễn ma trận như một phép biến đổi gồm:\n1. Xoay hệ trục đầu vào (\\( V \\))\n2. Co giãn dọc các trục toạ độ (\\( \\Sigma \\))\n3. Xoay hệ trục đầu ra (\\( U \\))\n\n## Ứng dụng\n- Giảm nhiễu và nén ảnh\n- Giải hệ phương trình không xác định\n- Phân tích dữ liệu trong học máy\n\n## Quan hệ với trị riêng\nGiá trị kỳ dị là căn bậc hai của trị riêng của \\( A^TA \\) hoặc \\( AA^T \\)\n\n## Tính chất\n- Mọi ma trận đều có SVD\n- Giá trị kỳ dị luôn không âm\n- Số giá trị k

## CHUNKING

In [5]:
headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            #("###", "Header 3"),
            #("####", "Header 4"),
            ]

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on = headers_to_split_on
)



# Split the documents into chunks based on the headers

all_chunks = []
for doc in documents:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        # Gộp metadata vào nội dung để vector hóa
        header1 = chunk.metadata.get("Header 1", "")
        header2 = chunk.metadata.get("Header 2", "")
        merged_text = f"[{header1}] [{header2}]\n{chunk.page_content}"

        # Gán lại nội dung mới
        chunk.page_content = merged_text
        chunk.metadata["source"] = doc.metadata.get("source", "")
        all_chunks.append(chunk)



#chunks = splitter.split_text(all_chunks)


In [7]:
len(all_chunks)

88

## EMBEDING

In [8]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True}
)


if os.path.exists(persist_directory):
    print(f"Persist directory {persist_directory} already exists. Loading existing vectorstore.")
    vectorstore = Chroma(persist_directory=persist_directory,
                            embedding_function=embedding_model,
                          collection_name="math_vectors")
    #vectorstore.add_documents(all_chunks)
    BATCH_SIZE = 100
    for i, chunk in enumerate(chunked(all_chunks, BATCH_SIZE)):
        print(f"Processing chunk {i + 1}/{len(all_chunks) // BATCH_SIZE + 1}")
        vectorstore.add_documents(chunk)
else:
    vectorstore = Chroma.from_documents(
        documents=all_chunks,
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_name="math_vectors"
    )




#vectorstore.persist()

C:\Users\dongh\AppData\Local\Temp\ipykernel_14208\156581409.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Test

In [5]:
from langchain.chains import RetrievalQA


embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model,
    collection_name="math_vectors"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "Cho 1 ví dụ về ma trận chuyển cơ sở"
docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(docs):
    print(f"\n--- Kết quả {i+1} ---")
    print("Từ file:", doc.metadata.get("source", ""))
    print("header1", doc.metadata.get("Header 1", ""))
    print("header2", doc.metadata.get("Header 2", ""))
    print("header3", doc.metadata.get("Header 3", ""))
    print(doc.page_content)



--- Kết quả 1 ---
Từ file: ..\TEST\file_15_chuyen_co_so.md
header1 Chuyển cơ sở
header2 Ma trận chuyển cơ sở
header3 
[Chuyển cơ sở] [Ma trận chuyển cơ sở]
Cho hai cơ sở \( B \) và \( C \), ma trận chuyển cơ sở từ \( B \) sang \( C \) là \( P \) sao cho:
\[ [v]_C = P [v]_B \]

--- Kết quả 2 ---
Từ file: ..\TEST\file_15_chuyen_co_so.md
header1 Chuyển cơ sở
header2 Ví dụ
header3 
[Chuyển cơ sở] [Ví dụ]
Chuyển vector \( v = (1, 2) \) từ cơ sở chuẩn sang cơ sở \( B = \{(1, 1), (1, -1)\} \)

--- Kết quả 3 ---
Từ file: ..\TEST\file_15_chuyen_co_so.md
header1 Chuyển cơ sở
header2 Tính chất
header3 
[Chuyển cơ sở] [Tính chất]
- Ma trận chuyển cơ sở là khả nghịch
- Ma trận biểu diễn của biến đổi tuyến tính thay đổi theo công thức:
\[ [T]_C = P [T]_B P^{-1} \]


In [ ]:
# Load model
model = SentenceTransformer(model_name, device=DEVICE)

In [ ]:
# Tạo embedding
texts = [chunk.page_content for chunk in chunks]
embeddings = model.encode(texts, 
                          convert_to_tensor=False,
                          normalize_embeddings=True)

In [10]:
embeddings

array([[ 0.01175354,  0.04333695,  0.01675817, ..., -0.01774279,
        -0.02854558,  0.07406582],
       [ 0.01095208,  0.0504429 , -0.01873817, ..., -0.02134096,
        -0.03130301,  0.03099393]], dtype=float32)